In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("PySpark").getOrCreate()

In [7]:
# Read a csv file into dataframe
emp = spark.read.format("csv").load("data/input/emp.csv")

In [8]:
emp.show()

+----------+---------+--------+--------------------+------------+--------------------+-------------------+-------------+------+
|       _c0|      _c1|     _c2|                 _c3|         _c4|                 _c5|                _c6|          _c7|   _c8|
+----------+---------+--------+--------------------+------------+--------------------+-------------------+-------------+------+
|EmployeeID|FirstName|LastName|               Email| PhoneNumber|            JobTitle|         Department|DateOfJoining|Salary|
|       101|     John|     Doe|john.doe@example.com|123-456-7890|   Software Engineer|                 IT|   2021-05-10| 85000|
|       102|     Jane|   Smith|jane.smith@exampl...|987-654-3210|      Data Scientist|     Data Analytics|   2019-11-20| 95000|
|       103|      Bob| Johnson|bob.johnson@examp...|555-123-4567|          HR Manager|    Human Resources|   2017-07-15| 78000|
|       104|    Alice|Williams|alice.williams@ex...|444-321-7654|Marketing Coordin...|          Marketin

In [9]:
emp.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)



In [10]:
emp = spark.read.format("csv").options(header=True).load("data/input/emp.csv")

In [11]:
emp.show()

+----------+---------+--------+--------------------+------------+--------------------+-------------------+-------------+------+
|EmployeeID|FirstName|LastName|               Email| PhoneNumber|            JobTitle|         Department|DateOfJoining|Salary|
+----------+---------+--------+--------------------+------------+--------------------+-------------------+-------------+------+
|       101|     John|     Doe|john.doe@example.com|123-456-7890|   Software Engineer|                 IT|   2021-05-10| 85000|
|       102|     Jane|   Smith|jane.smith@exampl...|987-654-3210|      Data Scientist|     Data Analytics|   2019-11-20| 95000|
|       103|      Bob| Johnson|bob.johnson@examp...|555-123-4567|          HR Manager|    Human Resources|   2017-07-15| 78000|
|       104|    Alice|Williams|alice.williams@ex...|444-321-7654|Marketing Coordin...|          Marketing|   2022-03-01| 60000|
|       105|  Charlie|   Brown|charlie.brown@exa...|333-456-7891|     Project Manager|         Operation

In [12]:
emp.printSchema()

root
 |-- EmployeeID: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- PhoneNumber: string (nullable = true)
 |-- JobTitle: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- DateOfJoining: string (nullable = true)
 |-- Salary: string (nullable = true)



In [14]:
# Let spark identify column datatype
option = {
    "header": True,
    "inferSchema": True
}
emp = spark.read.format("csv").options(**option).load("data/input/emp.csv")

In [15]:
emp.printSchema()

root
 |-- EmployeeID: integer (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- PhoneNumber: string (nullable = true)
 |-- JobTitle: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- DateOfJoining: date (nullable = true)
 |-- Salary: integer (nullable = true)



In [18]:
# Create schema
_schema = "EmployeeID int, FirstName string, LastName string, Email string, PhoneNumber string, JobTitle string, Department string, DateOfJoining date, Salary int"
emp = spark.read.format("csv").options(header=True).schema(_schema).load("data/input/emp.csv")

In [19]:
emp.printSchema()

root
 |-- EmployeeID: integer (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- PhoneNumber: string (nullable = true)
 |-- JobTitle: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- DateOfJoining: date (nullable = true)
 |-- Salary: integer (nullable = true)



In [36]:
# Handle bad records - PERMISSIVE (Default Mode)
_schema = "EmployeeID int, FirstName string, LastName string, Email string, PhoneNumber string, JobTitle string, Department string, DateOfJoining date, Salary int, _corrupt_record string"
emp_p = spark.read.format("csv").options(header=True).schema(_schema).load("data/input/emp_new.csv")

In [39]:
emp_p.where("_corrupt_record is null").show()

+----------+---------+--------+--------------------+------------+--------------------+-------------------+-------------+------+---------------+
|EmployeeID|FirstName|LastName|               Email| PhoneNumber|            JobTitle|         Department|DateOfJoining|Salary|_corrupt_record|
+----------+---------+--------+--------------------+------------+--------------------+-------------------+-------------+------+---------------+
|       101|     John|     Doe|john.doe@example.com|123-456-7890|   Software Engineer|                 IT|   2021-05-10| 85000|           NULL|
|       102|     Jane|   Smith|jane.smith@exampl...|987-654-3210|      Data Scientist|     Data Analytics|   2019-11-20| 95000|           NULL|
|       103|      Bob| Johnson|bob.johnson@examp...|555-123-4567|          HR Manager|    Human Resources|   2017-07-15| 78000|           NULL|
|       104|    Alice|Williams|alice.williams@ex...|444-321-7654|Marketing Coordin...|          Marketing|   2022-03-01| 60000|         

In [45]:
# Rename default _corrupt_record column
_schema = "EmployeeID int, FirstName string, LastName string, Email string, PhoneNumber string, JobTitle string, Department string, DateOfJoining date, Salary int, bad_record string"
emp_p_renamed = spark.read.format("csv").schema(_schema).option("columnNameOfCorruptRecord","bad_record").option("header",True).load("data/input/emp_new.csv")
emp_p_renamed.where("bad_record is null").show()

+----------+---------+--------+--------------------+------------+--------------------+-------------------+-------------+------+----------+
|EmployeeID|FirstName|LastName|               Email| PhoneNumber|            JobTitle|         Department|DateOfJoining|Salary|bad_record|
+----------+---------+--------+--------------------+------------+--------------------+-------------------+-------------+------+----------+
|       101|     John|     Doe|john.doe@example.com|123-456-7890|   Software Engineer|                 IT|   2021-05-10| 85000|      NULL|
|       102|     Jane|   Smith|jane.smith@exampl...|987-654-3210|      Data Scientist|     Data Analytics|   2019-11-20| 95000|      NULL|
|       103|      Bob| Johnson|bob.johnson@examp...|555-123-4567|          HR Manager|    Human Resources|   2017-07-15| 78000|      NULL|
|       104|    Alice|Williams|alice.williams@ex...|444-321-7654|Marketing Coordin...|          Marketing|   2022-03-01| 60000|      NULL|
|       105|  Charlie|   Br

In [46]:
# Handle bad records - DROPMALFORMED (Drops bad records directly)
_schema = "EmployeeID int, FirstName string, LastName string, Email string, PhoneNumber string, JobTitle string, Department string, DateOfJoining date, Salary int"
df_m = spark.read.format("csv").schema(_schema).option("header",True).option("mode","DROPMALFORMED").load("data/input/emp_new.csv")

In [47]:
df_m.show()

+----------+---------+--------+--------------------+------------+--------------------+-------------------+-------------+------+
|EmployeeID|FirstName|LastName|               Email| PhoneNumber|            JobTitle|         Department|DateOfJoining|Salary|
+----------+---------+--------+--------------------+------------+--------------------+-------------------+-------------+------+
|       101|     John|     Doe|john.doe@example.com|123-456-7890|   Software Engineer|                 IT|   2021-05-10| 85000|
|       102|     Jane|   Smith|jane.smith@exampl...|987-654-3210|      Data Scientist|     Data Analytics|   2019-11-20| 95000|
|       103|      Bob| Johnson|bob.johnson@examp...|555-123-4567|          HR Manager|    Human Resources|   2017-07-15| 78000|
|       104|    Alice|Williams|alice.williams@ex...|444-321-7654|Marketing Coordin...|          Marketing|   2022-03-01| 60000|
|       105|  Charlie|   Brown|charlie.brown@exa...|        NULL|     Project Manager|         Operation

In [48]:
# Handle bad records - FAILFAST (Fails job directly when encounter bad records)
_schema = "EmployeeID int, FirstName string, LastName string, Email string, PhoneNumber string, JobTitle string, Department string, DateOfJoining date, Salary int"
df_m = spark.read.format("csv").schema(_schema).option("header",True).option("mode","FAILFAST").load("data/input/emp_new.csv")

In [49]:
df_m.show()

Py4JJavaError: An error occurred while calling o229.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 25.0 failed 1 times, most recent failure: Lost task 0.0 in stage 25.0 (TID 25) (4f44685e94d2 executor driver): org.apache.spark.SparkException: [MALFORMED_RECORD_IN_PARSING.WITHOUT_SUGGESTION] Malformed records are detected in record parsing: [108,John,Doe,john.doe@example.com,123-456-7890,Software Engineer,IT,null,85000].
Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'. 
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1610)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:79)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:456)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.coll